<a href="https://colab.research.google.com/github/GomesVinicius/Curso-ML/blob/master/PreProcessamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose       import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/2024 1/ML/Course/1/heart_tratado.csv')

In [ ]:
df.head()

In [ ]:
df.shape

(917, 12)

# Tratamento nominal para categórico (Manual)

In [ ]:
df2 = pd.DataFrame.copy(df)

In [ ]:
df2['Sex'].replace({'M': 0, 'F': 1}, inplace=True)
df2['ChestPainType'].replace({'TA': 0, 'ATA': 1, 'NAP': 2, 'ASY': 3}, inplace=True)
df2['RestingECG'].replace({'Normal': 0, 'ST': 1, 'LVH': 2}, inplace=True)
df2['ExerciseAngina'].replace({'N': 0, 'Y': 1}, inplace=True)
df2['ST_Slope'].replace({'Up': 0, 'Flat':1, 'Down': 2}, inplace=True)

In [ ]:
df2.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,1,140,289.0,0,0,172,0,0.0,0,0
1,49,1,2,160,180.0,0,0,156,0,1.0,1,1
2,37,0,1,130,283.0,0,1,98,0,0.0,0,0
3,48,1,3,138,214.0,0,0,108,1,1.5,1,1
4,54,0,2,150,195.0,0,0,122,0,0.0,0,0


In [ ]:
df2.dtypes

Age                 int64
Sex                 int64
ChestPainType       int64
RestingBP           int64
Cholesterol       float64
FastingBS           int64
RestingECG          int64
MaxHR               int64
ExerciseAngina      int64
Oldpeak           float64
ST_Slope            int64
HeartDisease        int64
dtype: object

# Atributos Previsores

In [ ]:
previsores = df2.drop('HeartDisease', axis=1).values
# previsores = df2.iloc[:, 0:11].values

In [ ]:
previsores

array([[40. ,  0. ,  1. , ...,  0. ,  0. ,  0. ],
       [49. ,  1. ,  2. , ...,  0. ,  1. ,  1. ],
       [37. ,  0. ,  1. , ...,  0. ,  0. ,  0. ],
       ...,
       [57. ,  0. ,  3. , ...,  1. ,  1.2,  1. ],
       [57. ,  1. ,  1. , ...,  0. ,  0. ,  1. ],
       [38. ,  0. ,  2. , ...,  0. ,  0. ,  0. ]])

In [ ]:
alvo = df2['HeartDisease'].values
# alvo = df2.iloc[:, 11] #pandas.core.series.Series

In [ ]:
alvo

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

# Análise das escalas dos atributos

In [ ]:
df2.describe()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.00000,917.000000
mean,53.509269,0.210469,2.251908,132.540894,244.635389,0.233370,0.604144,136.789531,0.404580,0.886696,0.63795,0.552890
std,9.437636,0.407864,0.931502,17.999749,53.347125,0.423206,0.806161,25.467129,0.491078,1.066960,0.60727,0.497466
min,28.000000,0.000000,0.000000,80.000000,85.000000,0.000000,0.000000,60.000000,0.000000,-2.600000,0.00000,0.000000
25%,47.000000,0.000000,2.000000,120.000000,214.000000,0.000000,0.000000,120.000000,0.000000,0.000000,0.00000,0.000000
50%,54.000000,0.000000,3.000000,130.000000,244.635389,0.000000,0.000000,138.000000,0.000000,0.600000,1.00000,1.000000
75%,60.000000,0.000000,3.000000,140.000000,267.000000,0.000000,1.000000,156.000000,1.000000,1.500000,1.00000,1.000000
max,77.000000,1.000000,3.000000,200.000000,603.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.00000,1.000000


In [ ]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [ ]:
previsores_df = pd.DataFrame(previsores_esc)
previsores_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,-1.432206,-0.516309,-1.344701,0.414627,0.832075,-0.551733,-0.749818,1.383339,-0.824310,-0.831502,-1.051095
1,-0.478057,1.936826,-0.270580,1.526360,-1.212261,-0.551733,-0.749818,0.754736,-0.824310,0.106251,0.596519
2,-1.750256,-0.516309,-1.344701,-0.141240,0.719543,-0.551733,0.491306,-1.523953,-0.824310,-0.831502,-1.051095
3,-0.584074,1.936826,0.803541,0.303453,-0.574578,-0.551733,-0.749818,-1.131075,1.213136,0.575128,0.596519
4,0.052026,-0.516309,-0.270580,0.970493,-0.930931,-0.551733,-0.749818,-0.581047,-0.824310,-0.831502,-1.051095


In [ ]:
previsores_df.describe()

,0,1,2,3,4,5,6,7,8,9,10
count,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,917.000000,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02
mean,1.859654e-16,7.748558e-18,1.046055e-16,7.767929e-16,-1.869340e-16,4.649135e-17,0.000000,-5.114048e-16,-1.046055e-16,7.748558e-17,-3.874279e-17
std,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00
min,-2.704405e+00,-5.163086e-01,-2.418822e+00,-2.920572e+00,-2.994023e+00,-5.517333e-01,-0.749818,-3.016886e+00,-8.243101e-01,-3.269662e+00,-1.051095e+00
25%,-6.900904e-01,-5.163086e-01,-2.705801e-01,-6.971063e-01,-5.745784e-01,-5.517333e-01,-0.749818,-6.596226e-01,-8.243101e-01,-8.315022e-01,-1.051095e+00
50%,5.202558e-02,-5.163086e-01,8.035410e-01,-1.412398e-01,0.000000e+00,-5.517333e-01,-0.749818,4.755658e-02,-8.243101e-01,-2.688500e-01,5.965186e-01
75%,6.881250e-01,-5.163086e-01,8.035410e-01,4.146267e-01,4.194568e-01,-5.517333e-01,0.491306,7.547357e-01,1.213136e+00,5.751284e-01,5.965186e-01
max,2.490407e+00,1.936826e+00,8.035410e-01,3.749826e+00,6.721265e+00,1.812470e+00,1.732430,2.561971e+00,1.213136e+00,4.982571e+00,2.244132e+00


# LabelEncoder
## transformação de variáveis categóricas em numéricas

In [ ]:
previsores2 = df.copy()

In [ ]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289.0,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180.0,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283.0,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214.0,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195.0,0,Normal,122,N,0.0,Up,0


In [ ]:
# previsores2.drop('Sex', axis=1)
previsores2['Sex'] = LabelEncoder().fit_transform(df['Sex'])
previsores2['ChestPainType'] = LabelEncoder().fit_transform(df['ChestPainType'])
previsores2['RestingECG'] = LabelEncoder().fit_transform(df['RestingECG'])
previsores2['ExerciseAngina'] = LabelEncoder().fit_transform(df['ExerciseAngina'])
previsores2['ST_Slope'] = LabelEncoder().fit_transform(df['ST_Slope'])

In [ ]:
previsores2.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289.0,0,1,172,0,0.0,2,0
1,49,0,2,160,180.0,0,1,156,0,1.0,1,1
2,37,1,1,130,283.0,0,2,98,0,0.0,2,0
3,48,0,0,138,214.0,0,1,108,1,1.5,1,1
4,54,1,2,150,195.0,0,1,122,0,0.0,2,0


In [ ]:
previsores2.shape

(917, 12)

## ColumnTransformer

In [ ]:
previsores2.drop('HeartDisease', axis=1, inplace=True)

In [ ]:
previsores3 = ColumnTransformer(
    transformers=[('OneHot', OneHotEncoder(), ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'])],
    remainder='passthrough', verbose=True
    ).fit_transform(previsores2)

[ColumnTransformer] ........ (1 of 2) Processing OneHot, total=   0.0s
[ColumnTransformer] ..... (2 of 2) Processing remainder, total=   0.0s


In [ ]:
pd.DataFrame(previsores3).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,40.0,140.0,289.0,0.0,172.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,49.0,160.0,180.0,0.0,156.0,1.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,37.0,130.0,283.0,0.0,98.0,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,48.0,138.0,214.0,0.0,108.0,1.5
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,54.0,150.0,195.0,0.0,122.0,0.0


In [ ]:
previsores3.shape

(917, 20)

## Escalonamento Again

In [ ]:
previsores3_esc = StandardScaler().fit_transform(previsores3)

In [ ]:
previsores3_df = pd.DataFrame(previsores3_esc)

In [ ]:
previsores3_df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
count,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,917.000000,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,917.000000,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02
mean,1.084798e-16,-1.472226e-16,1.937139e-17,-3.874279e-17,3.874279e-17,6.973702e-17,0.000000,-9.298269e-17,1.549712e-17,-4.261707e-17,4.261707e-17,8.523413e-17,0.000000,-3.874279e-17,1.859654e-16,7.884157e-16,3.014189e-15,-1.549712e-17,-5.114048e-16,-1.859654e-16
std,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00
min,-5.163086e-01,-1.936826e+00,-1.085425e+00,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,-1.226974e+00,-4.907810e-01,-1.213136e+00,-8.243101e-01,-2.716072e-01,-1.001091,-8.698879e-01,-2.704405e+00,-2.920572e+00,-2.994023e+00,-5.517333e-01,-3.016886e+00,-3.269662e+00
25%,-5.163086e-01,5.163086e-01,-1.085425e+00,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,-1.226974e+00,-4.907810e-01,-1.213136e+00,-8.243101e-01,-2.716072e-01,-1.001091,-8.698879e-01,-6.900904e-01,-6.971063e-01,-5.745784e-01,-5.517333e-01,-6.596226e-01,-8.315022e-01
50%,-5.163086e-01,5.163086e-01,9.212982e-01,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,8.150134e-01,-4.907810e-01,8.243101e-01,-8.243101e-01,-2.716072e-01,0.998910,-8.698879e-01,5.202558e-02,-1.412398e-01,3.198360e-15,-5.517333e-01,4.755658e-02,-2.688500e-01
75%,-5.163086e-01,5.163086e-01,9.212982e-01,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,8.150134e-01,-4.907810e-01,8.243101e-01,1.213136e+00,-2.716072e-01,0.998910,1.149573e+00,6.881250e-01,4.146267e-01,4.194568e-01,-5.517333e-01,7.547357e-01,5.751284e-01
max,1.936826e+00,5.163086e-01,9.212982e-01,2.073784e+00,1.881384e+00,4.351412e+00,1.969177,8.150134e-01,2.037569e+00,8.243101e-01,1.213136e+00,3.681787e+00,0.998910,1.149573e+00,2.490407e+00,3.749826e+00,6.721265e+00,1.812470e+00,2.561971e+00,4.982571e+00


# Treino & Teste

**alvo** = variável que se pretende atingir (tem ou não doença cardíaca).

**previsores** = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas manualmente, sem escalonar.

**previsores_esc** = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas, escalonada.

**previsores2** = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas pelo labelencoder.

**previsores3** = conjunto de variáveis previsoras transformadas pelo labelencoder e onehotencoder, sem escalonar.

**previsores3_esc** = conjunto de variáveis previsoras transformadas pelo labelencoder e onehotencoder escalonada.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    previsores2, alvo, test_size=0.3, random_state=0)

In [ ]:
y_train

array([1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,

# NAIVE BAYES

In [ ]:
naive = GaussianNB()
naive.fit(X_train, y_train)

GaussianNB()

In [ ]:
previsoes_naive = naive.predict(X_test)
previsoes_naive

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0])

In [ ]:
# accuracy_score(y_test, previsoes_naive)
accuracy_score(y_test, naive.predict(X_test))

0.8442028985507246

In [ ]:
confusion_matrix(y_test, previsoes_naive)

array([[102,  19],
       [ 24, 131]])

In [ ]:
print(classification_report(y_test, previsoes_naive))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83       121
           1       0.87      0.85      0.86       155

    accuracy                           0.84       276
   macro avg       0.84      0.84      0.84       276
weighted avg       0.85      0.84      0.84       276



## Dados de Treino

In [ ]:
previsoes_treino = naive.predict(X_train)

In [ ]:
accuracy_score(y_train, previsoes_treino)

0.8361934477379095

In [ ]:
confusion_matrix(y_train, previsoes_treino)

array([[243,  46],
       [ 59, 293]])

## Validação Cruzada

In [ ]:
#Separando os dados em folds
kfold = KFold(n_splits=30, shuffle=True, random_state=5)

In [ ]:
#CCriando o modelo
modelo = GaussianNB()
resultado = cross_val_score(modelo, previsores3_esc, alvo, cv=kfold)
resultado

array([0.87096774, 0.87096774, 0.90322581, 0.80645161, 0.77419355,
       0.90322581, 0.70967742, 0.87096774, 0.77419355, 0.87096774,
       0.90322581, 0.77419355, 0.93548387, 0.90322581, 0.87096774,
       0.87096774, 0.87096774, 0.93333333, 0.93333333, 0.9       ,
       0.76666667, 0.83333333, 0.8       , 0.7       , 0.83333333,
       0.93333333, 0.86666667, 0.9       , 0.86666667, 0.8       ])

In [ ]:
#média e desvio padrão
resultado.mean()

0.8516845878136201

Nas validações cruzadas, os resultados não podem diferir muito comparado com a acurácia do modelo.
No caso do modelo, a acurácia foi de *85,42%*
No caso da validação cruzada, a acurácia foi de *85,16*

# SVM

In [ ]:
svm = SVC(kernel='rbf', C=2, random_state=1)

In [ ]:
svm.fit(X_train, y_train)

SVC(C=2, random_state=1)

In [ ]:
previsoes_svm = svm.predict(X_train)

In [ ]:
accuracy_score(y_train, previsoes_svm)

0.9282371294851794

In [ ]:
confusion_matrix(y_train, previsoes_svm)

array([[257,  32],
       [ 14, 338]])

In [ ]:
print(classification_report(y_train, previsoes_svm))

              precision    recall  f1-score   support

           0       0.95      0.89      0.92       289
           1       0.91      0.96      0.94       352

    accuracy                           0.93       641
   macro avg       0.93      0.92      0.93       641
weighted avg       0.93      0.93      0.93       641



## Análise dados de treino

In [ ]:
previsoes_treino = svm.predict(X_train)

In [ ]:
accuracy_score(y_train, previsoes_treino)

0.9282371294851794

In [ ]:
confusion_matrix(y_train, previsoes_treino)

array([[257,  32],
       [ 14, 338]])

## Validação Cruzada

In [ ]:
kfold = KFold(n_splits=30, shuffle=True, random_state=5)

In [ ]:
modelo = SVC(kernel='rbf', C=2, random_state=1)
result = cross_val_score(modelo, previsores3_esc, alvo, cv=kfold)
result

array([0.96774194, 0.83870968, 0.87096774, 0.90322581, 0.83870968,
       0.87096774, 0.70967742, 0.87096774, 0.80645161, 0.80645161,
       0.87096774, 0.80645161, 0.96774194, 0.90322581, 0.80645161,
       0.83870968, 0.80645161, 0.93333333, 0.86666667, 0.93333333,
       0.83333333, 0.86666667, 0.86666667, 0.63333333, 0.83333333,
       0.93333333, 0.9       , 0.86666667, 0.9       , 0.86666667])

In [ ]:
result.mean()

0.8572401433691755

# Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistica = LogisticRegression(random_state=1, max_iter=500
                               , penalty='l2', tol=0.0001, C=1,
                               solver='lbfgs')

In [ ]:
logistica.fit(X_train, y_train);

In [ ]:
logistica.intercept_

array([0.38745198])

In [ ]:
logistica.coef_

array([[ 0.0123781 , -1.27430395,  0.66527539,  0.00315881,  0.00427309,
         0.92061278,  0.09485448, -0.01172447,  1.02708424,  0.24925175,
         1.64750739]])

In [ ]:
previsoes_logistica = logistica.predict(X_test)

In [ ]:
accuracy_score(y_test, previsoes_logistica)

0.8623188405797102

In [ ]:
confusion_matrix(y_test, previsoes_logistica)

array([[ 99,  22],
       [ 18, 137]])

In [ ]:
confusion_matrix(y_test, previsoes_logistica)

array([[ 97,  24],
       [ 17, 138]])

In [ ]:
print(classification_report(y_test, previsoes_logistica))

              precision    recall  f1-score   support

           0       0.85      0.80      0.83       121
           1       0.85      0.89      0.87       155

    accuracy                           0.85       276
   macro avg       0.85      0.85      0.85       276
weighted avg       0.85      0.85      0.85       276



- variável: *previsores*
 - **lbfgs com l2**: 82.97%
 - **newton-cg com l2**: 85.50%

- variável: *previsores_esc*
 - **lbfgs com l2 com max_iter=500**: 85.69%
 - **lbfgs com l2 com max_iter=100**: 85.86%

- variável: *previsores3*
 - **lbfgs com l2 com max_iter=1000**: 86.23%

- variável: *previsores3_esc*
 - **lbfgs com l2 com max_iter=500**: 86.23% 🏆

## Análise dados de treino

In [ ]:
previsoes_treino = logistica.predict(X_train)

In [ ]:
accuracy_score(y_train, previsoes_treino)

0.858034321372855

In [ ]:
confusion_matrix(y_train, previsoes_treino)

array([[237,  52],
       [ 39, 313]])

## Validação Cruzada

In [ ]:
kfold = KFold(n_splits=30, shuffle=True, random_state=5)

In [ ]:
modelo = LogisticRegression(random_state=1, max_iter=500
                               , penalty='l2', tol=0.0001, C=1,
                               solver='lbfgs')

In [ ]:
result = cross_val_score(modelo, previsores3_esc, alvo, cv=kfold)

In [ ]:
result.mean()

0.8583154121863799

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10, metric='minkowski')
knn.fit(X_train, y_train);

In [ ]:
previsao_knn = knn.predict(X_test)

In [ ]:
accuracy_score(y_test, previsao_knn)

0.6811594202898551

In [ ]:
list = []

for x in range(3, 106):
  knn = KNeighborsClassifier(n_neighbors=x, metric='euclidean')
  knn.fit(X_train, y_train)

  accuracy = (accuracy_score(y_test, knn.predict(X_test)) * 100)

  print(x, accuracy)

  list.append(accuracy)

max(list)

3 81.88405797101449
4 80.79710144927536
5 82.6086956521739
6 83.33333333333334
7 84.05797101449275
8 83.69565217391305
9 84.42028985507247
10 84.05797101449275
11 85.5072463768116
12 85.5072463768116
13 84.78260869565217
14 85.5072463768116
15 85.14492753623189
16 84.42028985507247
17 85.14492753623189
18 85.14492753623189
19 85.86956521739131
20 85.5072463768116
21 85.5072463768116
22 85.14492753623189
23 85.14492753623189
24 84.78260869565217
25 85.14492753623189
26 85.14492753623189
27 85.5072463768116
28 84.42028985507247
29 85.14492753623189
30 84.42028985507247
31 84.78260869565217
32 84.42028985507247
33 84.78260869565217
34 84.42028985507247
35 84.78260869565217
36 84.42028985507247
37 84.42028985507247
38 84.78260869565217
39 84.78260869565217
40 84.78260869565217
41 84.78260869565217
42 84.42028985507247
43 84.42028985507247
44 84.05797101449275
45 84.05797101449275
46 84.42028985507247
47 84.42028985507247
48 84.05797101449275
49 84.05797101449275
50 84.05797101449275
51 83.

85.86956521739131

In [ ]:
# Tentando automatizar
# Uma forma de não precisar dar um ctrl f para ver o número de vizinhos de maior número
# pegando somente a maior porcentagem

# Ainda não funciona :(
list = pd.DataFrame([{'posicao': 0, '%': 0}])
pd.concat([list, pd.DataFrame([{'posicao': 1, '%': 1}])], ignore_index=True)
print(list)
for x in range(27, 33):
  knn = KNeighborsClassifier(n_neighbors=x, metric='minkowski')
  knn.fit(X_train, y_train)

  accuracy = (accuracy_score(y_test, knn.predict(X_test)) * 100)



  # list.update({'posicao': x, '%': accuracy})
  # list['posicao'] = x
  # list['%'] = accuracy

  print('Vizinhos {} - {:.2f}%'.format(x, accuracy))

print(list)

   posicao  %
0        0  0
Vizinhos 27 - 69.20%
Vizinhos 28 - 69.20%
Vizinhos 29 - 68.84%
Vizinhos 30 - 69.57%
Vizinhos 31 - 69.93%
Vizinhos 32 - 69.20%
   posicao  %
0        0  0


- **previsores**
 - 45 vizinhos: 71.37% (euclidean)
 - 23 vizinhos: 73.55% (cityblock)

- **previsores_esc**
 - 19 vizinhos: 85.86% (euclidean)
 - 17 vizinhos: 87.31% (cityblock)

- **previsores2**
 - péssimos resultados

- **previsores3**
  - péssimos resultados

- **previsores3_esc**
  - péssimos resultados

In [ ]:
knn = KNeighborsClassifier(n_neighbors=17, metric='cityblock')
knn.fit(X_train, y_train)
previsao_knn = knn.predict(X_test)

(accuracy_score(y_test, knn.predict(X_test)) * 100)

87.31884057971014

In [ ]:
confusion_matrix(y_test, previsao_knn)

array([[103,  18],
       [ 17, 138]])

## Avaliação dos dados de treino

In [ ]:
previsoes_treino = knn.predict(X_train)

In [ ]:
accuracy_score(y_train, previsoes_treino)

0.8673946957878315

In [ ]:
confusion_matrix(y_train, previsoes_treino)

array([[242,  47],
       [ 38, 314]])

## Validação cruzada

In [ ]:
kfold = KFold(n_splits=30, shuffle=True, random_state=5)

In [ ]:
modelo = KNeighborsClassifier(n_neighbors=17, metric='cityblock')
result = cross_val_score(modelo, previsores_esc, alvo, cv=kfold)

result.mean()

0.8592831541218636

# Árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
arvore = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=3)
arvore.fit(X_train, y_train);

In [ ]:
previsores_arvore = arvore.predict(X_test)

In [ ]:
print(arvore.class_weight)

None


## Acurácia

In [ ]:
accuracy_score(y_test, previsores_arvore)

0.782608695652174

In [ ]:
confusion_matrix(y_test, previsores_arvore)

array([[ 98,  23],
       [ 22, 133]])

In [ ]:
print(classification_report(y_test, previsores_arvore))

              precision    recall  f1-score   support

           0       0.82      0.81      0.81       121
           1       0.85      0.86      0.86       155

    accuracy                           0.84       276
   macro avg       0.83      0.83      0.83       276
weighted avg       0.84      0.84      0.84       276



- previsores
  - Diminuindo a profundidade (max_depth) o overfitting diminui
    - Entropy: 81.15% (aparente overfitting)
    - Gini: 76.44% (aparente overfitting)

- previsores_esc (max_depth = 3)
  - Entropy: 83.69%
  - Gini: 83.69%

- previsores2, previsores3, previsores3_esc
  - Entropy 83.69%

## Análise dados de treino

In [ ]:
previsoes_treino = arvore.predict(X_train)

In [ ]:
accuracy_score(y_train, previsoes_treino)

1.0

In [ ]:
confusion_matrix(y_train, previsoes_treino)

array([[289,   0],
       [  0, 352]])

## Validação Cruzada

In [ ]:
kfold = KFold(n_splits=30, shuffle=True, random_state=5)

In [ ]:
modelo = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=3)
result = cross_val_score(modelo, previsores, alvo, cv=kfold)

result.mean()

0.8342293906810034

# Random Forest

In [ ]:
#Ensemble = É o resultado de um aglomerado de algoritmos executados
#Podendo pegar a Moda para problemas de classificação ou Média para Regressão
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random = RandomForestClassifier(n_estimators=150, criterion='entropy',
                                max_depth=4, random_state=0)
random.fit(X_train, y_train);

In [ ]:
previsoes_random = random.predict(X_test)

## Acurácia

In [ ]:
accuracy_score(y_test, previsoes_random)

0.8659420289855072

In [ ]:
confusion_matrix(y_test, previsoes_random)

array([[100,  21],
       [ 20, 135]])

In [ ]:
print(classification_report(y_test, previsoes_random))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83       121
           1       0.87      0.85      0.86       155

    accuracy                           0.84       276
   macro avg       0.84      0.84      0.84       276
weighted avg       0.85      0.84      0.84       276



## Análise dados de treino

In [ ]:
previsoes_treino = random.predict(X_train)

In [ ]:
accuracy_score(y_train, previsoes_treino)

0.890795631825273

In [ ]:
(0.8659420289855072 - 0.890795631825273) *100

-2.4853602839765787

In [ ]:
confusion_matrix(y_train, previsoes_treino)

array([[239,  50],
       [ 23, 329]])

## Validação Cruzada


In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=5)

In [ ]:
modelo = RandomForestClassifier(n_estimators=150, criterion='entropy',
                                max_depth=4, random_state=0)

In [ ]:
resultado = cross_val_score(modelo, previsores, alvo, cv=kfold)
resultado.mean()

0.8604359705393205

## Resultados

- previsores
  - max_depth=4: *85.14%*
  - max_depth=3: *84.42%*
  - max_depth=4 & n_estimators=150 = *86.59%* 🏆

- todas outras variáveis não obtiveram bons resultados

# XGBOOST

In [ ]:
from xgboost import XGBClassifier

In [189]:
xg = XGBClassifier(max_depth=2, learning_rate=0.05, n_estimators=130
                   ,objective='binary:logistic', random_state=3)
xg.fit(X_train, y_train);

In [190]:
previsoes_xg = xg.predict(X_test)

## Acurácia

In [191]:
accuracy_score(y_test, previsoes_xg)

0.8586956521739131

In [ ]:
confusion_matrix(y_test, previsoes_xg)

array([[101,  20],
       [ 19, 136]])

In [ ]:
print(classification_report(y_test, previsoes_xg))

## Análise dados de treino

In [192]:
previsoes_treino = xg.predict(X_train)

In [193]:
accuracy_score(y_train, previsoes_treino)

0.9001560062402496

In [ ]:
confusion_matrix(y_train, previsoes_treino)

array([[260,  29],
       [ 42, 310]])

In [194]:
(0.8586956521739131 - 0.9001560062402496) * 100

-4.146035406633652

## Validação cruzada

In [ ]:
kfold = KFold(n_splits=30, shuffle=True, random_state=5)

In [ ]:
modelo = XGBClassifier(max_depth=2, learning_rate=0.05, n_estimators=250
                   ,objective='binary:logistic', random_state=3)

resultado = cross_val_score(modelo, previsores, alvo, cv=kfold)

In [ ]:
resultado.mean()

0.8702150537634407

## Resultado

- previsores (binary:logistic)
 - max_depth=2, learning_rate=0.05, n_estimators=130: *85.86%* (4.3 pontos de diferença) 🏆

- previsores (binary:logistic)
 - Não se mostrou mais eficaz que o binary:logistic

# LIGHTGBM

In [ ]:
!pip install lightgbm

In [ ]:
import lightgbm as lgb

In [ ]:
dataset = lgb.Dataset(X_train, label=y_train)

In [ ]:
parametros = {
    'num_leaves': 151,
    'objective': 'binary',
    'max-depth': 2,
    'learning_rate': .02,
    'max_bin': 200
}

In [ ]:
lgbm = lgb.train(parametros, dataset, num_boost_round=150);

[LightGBM] [Warning] Unknown parameter: max-depth
[LightGBM] [Warning] Unknown parameter: max-depth
[LightGBM] [Warning] Unknown parameter: max-depth
[LightGBM] [Info] Number of positive: 352, number of negative: 289
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000097 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 405
[LightGBM] [Info] Number of data points in the train set: 641, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.549142 -> initscore=0.197204
[LightGBM] [Info] Start training from score 0.197204
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [ ]:
previsoes_lgbm = lgbm.predict(X_test)

In [ ]:
for i in range (0, previsoes_lgbm.shape[0]):
  if previsoes_lgbm[i] >= 0.5:
    previsoes_lgbm[i] = 1
  else:
    previsoes_lgbm[i] = 0

In [ ]:
previsoes_lgbm

array([1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1.,
       0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
       1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1.,
       0., 1., 1., 1., 1.

## Acurácia

In [ ]:
accuracy_score(y_test, previsoes_lgbm)

0.855072463768116

## Análise dados de treino

In [ ]:
previsoes_treino = lgbm.predict(X_train)

In [ ]:
for i in range (0, previsoes_treino.shape[0]):
  if previsoes_treino[i] >= 0.5:
    previsoes_treino[i] = 1
  else:
    previsoes_treino[i] = 0

In [ ]:
accuracy_score(y_train, previsoes_treino)

0.9344773790951638

## Validação cruzada

In [ ]:
kfold = KFold(n_splits=30, shuffle=True, random_state=5)

In [ ]:
modelo = lgb.LGBMClassifier(num_leaves=151, objective='binary',
    max_depth=2, learning_rate=.02, max_bin= 200)

resultado = cross_val_score(modelo, previsores, alvo, cv=kfold)

In [ ]:
resultado.mean()

0.8483512544802866

## Resultado

- previsores
 -  'num_leaves': 151,
    'objective': 'binary',
    'max-depth': 2,
    'learning_rate': .02,
    'max_bin': 200'
    *85.50%* 🏆

# Catboost

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.5 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

In [ ]:
previsores_catboost = df.drop('HeartDisease', axis=1)
alvo_catboost = df['HeartDisease']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    previsores_catboost, alvo_catboost, test_size=0.3, random_state=0
    )

In [ ]:
categoricas = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [ ]:
catboost = CatBoostClassifier(task_type='CPU', iterations=120, learning_rate=0.1
                              ,depth=8, random_state=5, eval_metric='Accuracy')

In [ ]:
catboost.fit(
    X_train, y_train, cat_features=categoricas
    ,plot=True, eval_set=(X_test, y_test)
    )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8861154	test: 0.7789855	best: 0.7789855 (0)	total: 6.51ms	remaining: 775ms
1:	learn: 0.8689548	test: 0.7934783	best: 0.7934783 (1)	total: 12.7ms	remaining: 747ms
2:	learn: 0.8471139	test: 0.7971014	best: 0.7971014 (2)	total: 15.3ms	remaining: 597ms
3:	learn: 0.8549142	test: 0.8043478	best: 0.8043478 (3)	total: 24.1ms	remaining: 699ms
4:	learn: 0.8517941	test: 0.8043478	best: 0.8043478 (3)	total: 30.3ms	remaining: 696ms
5:	learn: 0.8486739	test: 0.8043478	best: 0.8043478 (3)	total: 35.6ms	remaining: 677ms
6:	learn: 0.8580343	test: 0.8115942	best: 0.8115942 (6)	total: 39.6ms	remaining: 639ms
7:	learn: 0.8642746	test: 0.8043478	best: 0.8115942 (6)	total: 42.8ms	remaining: 599ms
8:	learn: 0.8533541	test: 0.8043478	best: 0.8115942 (6)	total: 46.6ms	remaining: 575ms
9:	learn: 0.8611544	test: 0.8115942	best: 0.8115942 (6)	total: 50ms	remaining: 550ms
10:	learn: 0.8642746	test: 0.8079710	best: 0.8115942 (6)	total: 53.4ms	remaining: 529ms
11:	learn: 0.8658346	test: 0.8043478	best: 0

## Análise dados de treino

In [ ]:
previsoes_catboost_treino = catboost.predict(X_train)

In [ ]:
accuracy_score(y_train, previsoes_catboost_treino)

0.9173166926677067

## Validação cruzada

In [ ]:
kfold = KFold(n_splits=30, shuffle=True, random_state=5)

In [ ]:
modelo = CatBoostClassifier(task_type='CPU', iterations=120, learning_rate=0.1
                              ,depth=8, random_state=5, eval_metric='Accuracy')

resultado = cross_val_score(modelo, previsores, alvo, cv=kfold)

0:	learn: 0.8803612	total: 4.77ms	remaining: 567ms
1:	learn: 0.8758465	total: 17.6ms	remaining: 1.04s
2:	learn: 0.8837472	total: 23.1ms	remaining: 899ms
3:	learn: 0.8927765	total: 32.6ms	remaining: 946ms
4:	learn: 0.8792325	total: 35.9ms	remaining: 825ms
5:	learn: 0.8803612	total: 40.4ms	remaining: 767ms
6:	learn: 0.8893905	total: 47.2ms	remaining: 763ms
7:	learn: 0.8860045	total: 58.8ms	remaining: 823ms
8:	learn: 0.8927765	total: 72.8ms	remaining: 898ms
9:	learn: 0.8950339	total: 79.6ms	remaining: 876ms
10:	learn: 0.8950339	total: 94.3ms	remaining: 934ms
11:	learn: 0.8961625	total: 101ms	remaining: 905ms
12:	learn: 0.8972912	total: 106ms	remaining: 872ms
13:	learn: 0.8972912	total: 117ms	remaining: 886ms
14:	learn: 0.8995485	total: 126ms	remaining: 885ms
15:	learn: 0.9006772	total: 133ms	remaining: 862ms
16:	learn: 0.9029345	total: 145ms	remaining: 878ms
17:	learn: 0.9040632	total: 152ms	remaining: 861ms
18:	learn: 0.9018059	total: 155ms	remaining: 824ms
19:	learn: 0.9051919	total: 16

In [ ]:
resultado.mean()

0.8810394265232974

In [ ]:
from scipy import stats

In [ ]:
stats.mode(resultado)

ModeResult(mode=0.8709677419354839, count=6)

## Resultado

- previsores catboost
 - *86.95%*

# Resultado Geral

- Naive Bayes = 85.42%
- SVM = 85.57%
- Regressão Logística = 86.23%
- KNN = 85.86%
- Árvore de Decisão = 83.69%
- Random Forest = 86.59%
- XGBOOST = 85.86%
- LGBM = 85.50%
- **CatBoost = 86.95%** 🏆

# GRID SEARCH (Otimização de Hiperparâmetros)

Buscando um melhor resultado no algoritmo XGBOOST (85.86% de acurácia) utilizando GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(random_state=0)

In [ ]:
param_grid = dict(
    max_depth=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    learning_rate=[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1],
    n_estimators=[50, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 320, 340, 360, 380, 400],
    objective=['binary:logistic', 'binary:logitraw', 'binary:hinge']
    )

In [ ]:
len(param_grid['max_depth']) * len(param_grid['learning_rate']) * len(param_grid['n_estimators']) * len(param_grid['objective'])


5400

In [ ]:
grid_search = GridSearchCV(xgb, param_grid, scoring='roc_auc', cv=4)

In [ ]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=4,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=0, ...),
             param_grid={'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06,
                                           0.07, 0.08, 0.09, 0.1],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [50, 80, 100, 120, 140, 160, 180, 200,
                                          220, 240, 260, 280, 300, 320, 340,
                                          360, 380, 400],
                         'objective': ['binary:logistic', 'binary:logitraw',
                                       'binary:hinge']},
             scoring='roc_auc')

## Melhores hiperparâmetros

In [ ]:
grid_search.best_params_

{'learning_rate': 0.09,
 'max_depth': 2,
 'n_estimators': 80,
 'objective': 'binary:logistic'}

In [ ]:
results_grid = pd.DataFrame(grid_search.cv_results_)

In [ ]:
results_grid.sort_values(by='mean_test_score', ascending=False, inplace=True)

results_grid.reset_index(drop=True, inplace=True)

results_grid[['param_max_depth', 'param_learning_rate', 'param_n_estimators',
    'mean_test_score', 'param_objective', 'std_test_score']].head()

,param_max_depth,param_learning_rate,param_n_estimators,mean_test_score,param_objective,std_test_score
0,2,0.09,80,0.926071,binary:logitraw,0.017340
1,2,0.09,80,0.926071,binary:logistic,0.017340
2,2,0.07,100,0.925518,binary:logitraw,0.017199
3,2,0.07,100,0.925518,binary:logistic,0.017199
4,2,0.1,80,0.925428,binary:logistic,0.015953


## Acurácia antigos hiperparâmetros


In [ ]:
xg = XGBClassifier(max_depth=2, learning_rate=0.05, n_estimators=250
                   ,objective='binary:logistic', random_state=3)
xg.fit(X_train, y_train);

accuracy_score(y_test, xg.predict(X_test))

0.8586956521739131

# Acurácia novos hiperparâmetros

In [ ]:
xg = XGBClassifier(
    max_depth=2, learning_rate=0.09, n_estimators=80,
    objective='binary:logitraw', random_state=0
    )
xg.fit(X_train, y_train)

accuracy_score(y_test, xg.predict(X_test))

0.8768115942028986

In [ ]:
accuracy_score(y_train, xg.predict(X_train))

0.8892355694227769

In [ ]:
(0.8586956521739131 - 0.8768115942028986) * 100

-1.8115942028985477

In [ ]:
(0.8768115942028986 - 0.8892355694227769) * 100

-1.242397521987837

## Resultado

A busca por melhores hiperparâmetros aumentou a acurácia do modelo em 1.18.

Deixando o modelo treinado com os dados de teste mais próximos dos resultados do modelo de treino, diminuindo possibilidade de overfitting. Diferença inicial de 4.14 para 1.24.

Modificando somente os hiperparâmetros 'learning_rate', 'max_depth', 'n_estimators' e 'objective'.
Esta é melhor configuração utilizando XGBoost para esta base de dados:

```
  {
    'learning_rate': 0.09,
    'max_depth': 2,
    'n_estimators': 80,
    'objective': 'binary:logistic'
  }
```

**Resumo**
 - Melhor algoritmo
  - Catboost: 86.95%

 - Antiga pontuação de XGBoost
  - 85.86%

 - Nova pontuação de XGBoost
  - 87.68% 🏆

Com este estudo no algoritmo XGBoost ele se torna o melhor modelo por 0.73. Com estudos similares no algoritmo Catboost, o resultado pode mudar.